<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/notebook_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Das Projekt aus dem Github klonen und in den Projektsordner**

In [21]:
#wenn die Ordner noch nicht geklont ist, soll dieser Fehler zuerst durchgeführt werden.
#!git clone https://github.com/hanhluukim/replication-topic-modelling-in-embedding-space.git

In [22]:
cd /content/replication-topic-modelling-in-embedding-space

/content/replication-topic-modelling-in-embedding-space


#**Die benötige Paketen für das Projekt mittels requirements.txt installieren**



In [23]:
# Falls die Packages noch nicht installiert wurden, 
!pip install -r "/content/replication-topic-modelling-in-embedding-space/requirements.txt"

# **Gebrauchte Paketen importieren**

In [24]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import umap.umap_ as umap
import time
import plotly.express as px
from sklearn import cluster
from sklearn import metrics

# **Vorverarbeitung und BOW-Repräsentationen für Textdaten durchführen**
1. Vocabular erstellen
2. BOW-Repräsentationen für allen Teildatensätzen

In [25]:
from src.preprare_dataset import TextDataLoader

In [26]:
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")
# Beispiel von Textdaten
textsloader.show_example_raw_texts(n_docs=2)

loading texts: ...
finished load!
check some sample texts of the dataset
['From', ':', 'lerxst', '@', 'wam', '.', 'umd', '.', 'edu', '(', "where's", 'my', 'thing', ')', 'Subject', ':', 'WHAT', 'car', 'is', 'this', '!', '?', 'Nntp', 'Posting', 'Host', ':', 'rac3', '.', 'wam', '.', 'umd', '.', 'edu', 'Organization', ':', 'University', 'of', 'Maryland', ',', 'College', 'Park', 'Lines', ':', '15', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2', 'door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s', '/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs'

In [27]:
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)

start: preprocessing: ...
finised: preprocessing!


In [28]:
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=10, stopwords_remove_from_voca=True)

permuted indices for the train set: [  2 201  54 139  75 119  93  35 189 128  27 101   8 109  71]
start creating vocabulary ...
length of the vocabulary: 348
sample ten words of the vocabulary: ['give', 'toronto', 'front', 'late', 'radio', 'ibm', 'coming', 'line', 'cost', 'remember']
length word2id list: 348
length id2word list: 348
finished: creating vocabulary


In [29]:
# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False 
word2id, id2word, train_set, test_set, val_set = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)

length train-documents-indices : 4803
length of the vocabulary: 348


start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 149
min doc-id: 0
all docs: 4803
all words: 4803
docidx unique 150
words unique: 348
ndocs: 150
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 1465
all words: 1465
docidx unique 50
words unique: 325
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 2, 4, 5, 7, 10, 12, 13, 14, 15]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 719
all words: 719
docidx unique 50
words unique: 240
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5

# **Vocabular und IDs anzeigen als Beispiel**

---





In [30]:
# show for samples: 100 word2id and id2 word
word2id_df_100 = pd.DataFrame()
word2id_df_100['word'] = list(word2id.keys())[:100]
word2id_df_100['id'] = list(word2id.values())[:100]
word2id_df_100

,word,id
0,give,0
1,toronto,1
2,front,2
3,late,3
4,radio,4
...,...,...
95,bible,95
96,de,96
97,case,97
98,windows,98


# **Die Größe von Datensätzen kontrollieren**

In [31]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train_set["tokens"])}')
print(f'Size of val set: {len(val_set["tokens"])}')
print(f'Size of test set: {len(test_set["test"]["tokens"])}')

Size of the vocabulary after prprocessing ist: 348
Size of train set: 150
Size of val set: 100
Size of test set: 50


# **Dokumenten wiederstellen für Word2Vec Embedding**

---



In [32]:
# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

,text-after-preprocessing
0,questions university engineering computer netw...
1,mark university writes making long fine car ma...
2,day article reply distribution world article c...
3,ca university love question bible long time qu...
4,mark newsreader tin version ibm wrote higher c...
...,...
95,jim car important software nntp posting host c...
96,life reply sun article late make remember arti...
97,chip technical article research writes key key...
98,internet nntp posting host cc write applicatio...


# **Word-Embedding trainieren mit dem Traindatensatz**

In [33]:
from src.embedding import WordEmbeddingCreator
save_path = Path.joinpath(Path.cwd(), "vocab_embedding.txt")
wb_creator = WordEmbeddingCreator(model_name="cbow", documents = docs_tr, save_path= save_path)
wb_creator.train(min_count=0, embedding_size= 10)
vocab = list(word2id.keys())
wb_creator.create_and_save_vocab_embedding(vocab, save_path)

word-embedding train begins
word-embedding train finished
length of vocabulary from word-embedding model 348
length of the vocabulary of prepraring-dataset-vocabulary: 348


100%|██████████| 348/348 [00:00<00:00, 71280.84it/s]


True

In [34]:
v = list(wb_creator.model.wv.vocab)[0]
vec = list(wb_creator.model.wv.__getitem__(v))
print(f'word-embedding of the word-- {v}: ')
print(f'vector: {vec}')
print(f'dim of vector: {len(vec)}')

word-embedding of the word-- questions: 
vector: [-0.08385208, 0.012325125, 0.09597363, 0.060030807, -0.015472969, 0.048684556, 0.045274347, 0.05470478, 0.06400303, -0.0026470025]
dim of vector: 10


# **Word-Embeddings visualieren als Beispiel**


In [35]:
# read word-embedding files
with open(save_path) as f:
  lines = f.readlines()
embedding_data = []
words_data = []
for t in lines:
  w = t.split("\t")[0]
  v = [float(e) for e in t.split("\t")[1].split(" ")]
  words_data.append(w)
  embedding_data.append(v)

In [36]:
# clustering words with KMeans and Words-Vectors
kmeans = cluster.KMeans(n_clusters=10)
kmeans.fit(embedding_data)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)

Cluster id labels for inputted data
[7 6 0 2 5 5 1 3 7 9 2 6 4 6 1 2 9 3 1 7 6 3 3 7 6 2 3 8 1 3 7 5 6 2 0 1 4
 1 2 3 4 9 4 1 8 7 6 5 6 9 4 5 6 6 4 2 4 6 9 2 9 9 2 4 0 5 2 9 7 1 3 1 6 3
 9 5 1 9 9 3 1 4 9 2 1 0 7 9 0 6 1 7 7 9 5 9 4 2 4 6 1 0 9 3 4 3 6 0 7 3 5
 1 9 7 1 9 1 9 3 0 9 0 5 1 3 6 7 1 2 2 6 0 7 6 2 1 3 4 7 2 9 7 6 2 1 9 9 6
 3 1 0 7 0 1 7 3 3 0 9 9 3 9 3 1 5 9 0 6 1 9 0 1 9 2 1 9 0 2 5 3 7 4 6 7 0
 6 7 1 6 1 3 1 5 1 9 5 5 2 1 6 1 1 0 0 1 3 1 7 2 1 7 5 3 2 7 7 7 4 7 1 0 7
 2 4 1 2 6 0 3 0 1 9 1 2 0 9 9 6 3 7 9 5 7 6 2 2 7 1 9 1 2 6 3 7 3 2 7 6 6
 0 0 7 6 1 7 1 5 1 7 5 9 3 7 2 1 7 3 3 3 4 1 3 9 6 1 9 1 1 7 0 5 1 1 1 6 7
 7 7 9 0 1 3 5 0 7 9 3 1 9 9 0 1 5 1 7 3 3 7 0 1 9 0 6 5 7 5 6 0 3 5 6 5 6
 5 0 0 7 0 1 2 0 5 5 0 5 0 6 7]
Centroids data
[[-0.02198676  0.0280378   0.09566523  0.00802669 -0.01447957  0.03926021
   0.00136707  0.05323057  0.04209634  0.00240668]
 [-0.10820695  0.06904454  0.21913771  0.04989188 -0.03500375  0.0968335
  -0.01803342  0.1417339   0.08738878 -0.008

In [37]:
# dimension reduction with umap
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(embedding_data)
print('Duration: {} seconds'.format(time.time() - start))

# show samples after dim-reduction in dataframe
wb = pd.DataFrame(embedding, columns=['x', 'y', 'z'])
wb['word'] = words_data
wb['cluster'] = ['cluster ' + str(c) for c in labels]

Duration: 2.0975961685180664 seconds


In [38]:
# visualization
fig = px.scatter_3d(wb, 
                    text = wb['word'],
                    x='x', y='y', z='z',
                    color = wb['cluster'],
                    title ="word-embedding-samples")
fig.show()

# **ETM-Model trainieren**

In [39]:
# using DocSet to use easier the modul DataSet from torch
from src.train_etm import DocSet, ETMTrain
from src.etm import ETM

vocab_size = len(list(word2id.keys()))
tr_set = DocSet("train", vocab_size, train_set)
print(len(tr_set))
print(tr_set.__getitem__(0))

150
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 2., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 2., 0., 0., 0., 0., 0., 0., 1., 0., 0., 2., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 

In [40]:
num_topics = 5
t_hidden_size = 100
rho_size = len(embedding_data[0])
emb_size = len(embedding_data[0])
theta_act = "relu"

class TrainArguments:
      def __init__(self, epochs, batch_size, log_interval):
          self.epochs = epochs
          self.batch_size = batch_size
          self.log_interval = log_interval

class OptimizerArguments:
      def __init__(self, optimizer_name, lr, wdecay):
            self.optimizer = optimizer_name
            self.lr = lr
            self.wdecay = wdecay
            
train_args = TrainArguments(epochs=10, batch_size=100, log_interval=None)
optimizer_args = OptimizerArguments(optimizer_name="Adam", lr=0.005, wdecay=0.1)

print(train_args.epochs)
print(optimizer_args.optimizer)

training_set = train_set

# define the ETM-model with setting-parameters
etm_model = ETM(
      num_topics, 
      vocab_size, 
      t_hidden_size, rho_size, emb_size, theta_act, 
      embedding_data, enc_drop=0.5)

# start training
train_class = ETMTrain().train(
    etm_model,
    vocab_size, 
    train_args, optimizer_args, training_set, 
    num_topics, t_hidden_size, rho_size, emb_size, theta_act, embedding_data, 0.5)

10
Adam
Defaulting to vanilla SGD
Epoch 0/10:
torch.Size([100])
torch.Size([])
total loss: 19716.6875
Epoch 1/10:
torch.Size([100])
torch.Size([])
total loss: 20000.634765625
Epoch 2/10:
torch.Size([100])
torch.Size([])
total loss: 19852.431640625
Epoch 3/10:
torch.Size([100])
torch.Size([])
total loss: 18369.73828125
Epoch 4/10:
torch.Size([100])
torch.Size([])
total loss: 20334.513671875
Epoch 5/10:
torch.Size([100])
torch.Size([])
total loss: 19058.90234375
Epoch 6/10:
torch.Size([100])
torch.Size([])
total loss: 20291.328125
Epoch 7/10:
torch.Size([100])
torch.Size([])
total loss: 17906.462890625
Epoch 8/10:
torch.Size([100])
torch.Size([])
total loss: 18501.6875
Epoch 9/10:
torch.Size([100])
torch.Size([])
total loss: 17745.640625
